In [1]:
import CNN_model
import torch
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pickle as pkl
from torch import Tensor
from joblib import dump, load
import submission
import pandas as pd

In [ ]:
print(torch.cuda.is_available())
torch.cuda.empty_cache()

In [2]:
cnn_model = CNN_model.CNN()

In [ ]:
model_step0 = cnn_model.main(loadname = 'lx_preprocessed_data0.joblib', pre_trained_model = None)
model_step1 = cnn_model.main(loadname = 'lx_preprocessed_data1.joblib', pre_trained_model = model_step0)
model_step2 = cnn_model.main(loadname = 'lx_preprocessed_data2.joblib', pre_trained_model = model_step1)

In [3]:
cnn_model = cnn_model.main(loadname = 'lx_preprocessed_data.joblib', pre_trained_model = None)

Starting epoch 1
Loss after mini-batch     1: 0.0213987797
Loss after mini-batch    11: 0.5175946856
Loss after mini-batch    21: 0.0057093226
Loss after mini-batch    31: 0.0039582049
Loss after mini-batch    41: 0.0020513223
Loss after mini-batch    51: 0.0009687775
Loss after mini-batch    61: 0.0007586151
Loss after mini-batch    71: 0.0006673849
Loss after mini-batch    81: 0.0008648684
Loss after mini-batch    91: 0.0005479341
Loss after mini-batch   101: 0.0004073409
Loss after mini-batch   111: 0.0004540162
Loss after mini-batch   121: 0.0004439252
Loss after mini-batch   131: 0.0007001483
Loss after mini-batch   141: 0.0005355409
Loss after mini-batch   151: 0.0003775057
Loss after mini-batch   161: 0.0004230013
Loss after mini-batch   171: 0.0006982020
Loss after mini-batch   181: 0.0006342489
Loss after mini-batch   191: 0.0003930753
Loss after mini-batch   201: 0.0003942552
Loss after mini-batch   211: 0.0004869558
Starting epoch 2
Loss after mini-batch     1: 0.0000552852


In [ ]:
cnn_model = cnn_model.main(loadname = 'lx_preprocessed_data1.joblib', pre_trained_model = None)

In [ ]:
cnn_model = cnn_model.main(loadname = 'lx_preprocessed_data2.joblib', pre_trained_model = None)

In [4]:
model_scripted = torch.jit.script(cnn_model) # Export to TorchScript
model_scripted.save('res50_pretrained_model.pt')

In [ ]:
predictions = cnn_model.pred(model_name='res50_pretrained_model.pt')

In [ ]:
dp = CNN_model.Data_Preprocessing()
data_train = CNN_model.load_images(path='./lazydata/', isTrain = True)
data_test = CNN_model.load_images(path='./lazydata/', isTrain = False)

In [ ]:
img0_array_test, img1_array_test, img2_array_test, depth_array_test, field_id_array = dp.tensorToArray(data=data_test, isTrain=False)
img0_array_train, img1_array_train, img2_array_train, depth_array_train, y_array = dp.tensorToArray(data=data_train, isTrain=True)

In [ ]:
# test
normalized_depth_test = dp.depth_normalization(depth=depth_array_test)

normalized_img0_test = dp.img_normalization(img=img0_array_test)
new_img_test = dp.combine_image_depth(img=normalized_img0_test, depth=normalized_depth_test)
ready_img_test = dp.reshape_data(new_img_test)

normalized_img1_test = dp.img_normalization(img=img1_array_test)
new_img1_test = dp.combine_image_depth(img=normalized_img1_test, depth=normalized_depth_test)
ready_img1_test = dp.reshape_data(new_img1_test)

normalized_img2_test = dp.img_normalization(img=img2_array_test)
new_img2_test = dp.combine_image_depth(img=normalized_img2_test, depth=normalized_depth_test)
ready_img2_test = dp.reshape_data(new_img2_test)

# train
normalized_depth_train = dp.depth_normalization(depth=depth_array_train)

normalized_img0_train = dp.img_normalization(img=img0_array_train)
new_img_train = dp.combine_image_depth(img=normalized_img0_train, depth=normalized_depth_train, whichImg = 0)
ready_img_train = dp.reshape_data(new_img_train)

normalized_img1_train = dp.img_normalization(img=img1_array_train)
new_img_train1 = dp.combine_image_depth(img=normalized_img1_train, depth=normalized_depth_train, whichImg = 1)
ready_img_train1 = dp.reshape_data(new_img_train1)

normalized_img2_train = dp.img_normalization(img=img2_array_train)
new_img_train2 = dp.combine_image_depth(img=normalized_img2_train, depth=normalized_depth_train, whichImg = 2)
ready_img_train2 = dp.reshape_data(new_img_train2)

In [ ]:
# test
testX = [ready_img_test, field_id_array]
testX_img1 = [ready_img1_test, field_id_array]
testX_img2 = [ready_img2_test, field_id_array]

# train
train_img0 = [ready_img_train, y_array]
train_img1 = [ready_img_train1, y_array]
train_img2 = [ready_img_train2, y_array]

In [ ]:
# test
dump(testX, 'preprocessed_testX.joblib')
dump(testX_img1, 'preprocessed_testX_img1.joblib')
dump(testX_img2, 'preprocessed_testX_img2.joblib')

# train
dump(train_img0, 'lx_preprocessed_data0.joblib')
dump(train_img1, 'lx_preprocessed_data1.joblib')
dump(train_img2, 'lx_preprocessed_data2.joblib')

In [ ]:
sub = submission.Submission()
df = sub.submit(filename = 'preprocessed_testX.joblib', modelname = 'res50_pretrained_model.pt')
df = sub.submit(filename = 'preprocessed_testX_img1.joblib', modelname = 'res50_pretrained_model_img1.pt')
df = sub.submit(filename = 'preprocessed_testX_img2.joblib', modelname = 'res50_pretrained_model_img2.pt')

df = sub.submit(filename = 'preprocessed_testX.joblib', modelname = 'res50_pretrained_model_combine.pt')
df = sub.submit(filename = 'preprocessed_testX_img1.joblib', modelname = 'res50_pretrained_model_combine.pt')
df = sub.submit(filename = 'preprocessed_testX_img2.joblib', modelname = 'res50_pretrained_model_combine.pt')

In [ ]:
print(df)